<html>
<table style="width:100%" align="center" border="0px white">
<tr></td>
<td><a href="https://colab.research.google.com/github/UCL-EO/uclgeog_msc_core/blob/master/NASA.ipynb">
         <img alt="Open In Colab" src="https://colab.research.google.com/assets/colab-badge.svg">
</a>
<td><a href="https://mybinder.org/v2/gh/UCL-EO/uclgeog_msc_core/master">
         <img alt="Binder" src="https://mybinder.org/badge_logo.svg">
</a></td>                               
<td><a href="https://github.com/UCL-EO/uclgeog_msc_core/blob/master/NASA.ipynb">
         <img alt="View in github" src="https://img.shields.io/static/v1?logo=github&label=View%20in&message=github&color=green">
</a></td>    
<td><a href="https://nbviewer.jupyter.org/github/UCL-EO/uclgeog_msc_core/blob/master/NASA.ipynb">
         <img alt="View in nbviewer" src="https://img.shields.io/static/v1?logo=jupyter&label=View%20in&message=nbviewer&color=blue">
</a></td>    
    </tr>
    </table>
    
<img alt="UCL" src="images/ucl_logo.png">
</html>

# NASA Code test


## Purpose 

In this notebook, we will test codes from `uclgeog_msc_core` for downloading and displaying NASA images. We will also be introducing and viewing the NASA MODIS LAI product. This notebook should serve as an introduction to accessing such products from [`NASA EarthData`](https://urs.earthdata.nasa.gov).

An additional purpose of this notebook is to test if the code associated with the setup of `uclgeog_msc_core` is working properly. 

We will also illustrate some of the things you will be able to do when you have finished this course. We use pre-generated codes in the library `uclgeog_msc_core` here, but in the course, we will *look under the bonnet* of such codes, and learn how to develop them.

## Prerequisites

Before you can use the material in this notebook, you will need to register as a user at the [`NASA EarthData`](https://urs.earthdata.nasa.gov/users/new).

Once you have done that, make sure you know your `username` and `password` ready for below.

## Credentials

We will store your credentials for [`NASA EarthData`] (https://urs.earthdata.nasa.gov/users/new) to allow easier data downloading. 

**N.B. using `cylog().login()` is only intended to work with access to NASA Earthdata and to prevent you having to expose your username and password in these notes**.


In the `uclgeog_msc_core` library, we have a Python class called `cylog`, written to allow easier persistent interface to NASA download servers.

First, we import `cylog` from the `uclgeog_msc_core` library.

Run the cell below:

In [1]:
from uclgeog_msc.cylog import cylog

If this gave an error, there is a problem importing the `uclgeog_msc_core` library and you shoiuld get help on this in a supoport class.

Assuming that worked, we can look at the `help` method for this class:

In [2]:
help(cylog)

Help on class cylog in module uclgeog_msc.cylog:

class cylog(builtins.object)
 |  cylog(init=False, destination_folder='.cylog')
 |  
 |  cylog provides a mechanism to partially hide username and
 |  password information that is required in plain text.
 |  
 |  It does this by storing a key and the encrypted version in
 |  a file accessible only to the user.
 |  
 |  Of course, when called (by the user) the (username, password)
 |  are exposed in plain text, so only use this when you 
 |  have to enter plain text username/password information.
 |  
 |  It is written as a utility to allow UCL MSc students to 
 |  show access to NASA Earthdata dataset download, without 
 |  the need to expose (username, password) in a submitted report.
 |  
 |  Stores (in a dictionary in ~/{dest_path}/.cylog.npz) an
 |  encrypted form of username and password (and key)
 |  
 |  Uses cryptography.fernet.Fernet() for encryption
 |  
 |  cylog().login() : returns plain text tuple
 |                    (use

This gives us a description of what it does and how it works, and shows how useful good documentation can be in our code.

Before you can use this, you will need to register at the [`NASA EarthData`](https://urs.earthdata.nasa.gov/users/new), as noted above. 

Make sure you know your `username` and `password` ready for below.

### Earthdata login

Run the cell below, and enter your `username` and `password` if prompted.

In [3]:
cy = cylog()

If you run this call again, it should no longer pronmpt you, as it will read the stored credentials.

In [4]:
cy = cylog()

If you want to force the code to let you re-enter your credentials (e.g. you got it wrong before, or have changed them), then change the call to:

    cy = cylog(init=True)
    
and re-run.

`cylog` stores your username and password in a file that only you can read. We can use this as a convenient way to pull some NASA MODIS data.

## MODIS LAI product 
To introduce geospatial processing, we will use a dataset from the MODIS LAI product over the UK. 

You should note that the dataset you need to use for your assessed practical is a MODIS dataset with similar characteristics to the one in this example.

The data product [MOD15](https://modis.gsfc.nasa.gov/data/dataprod/mod15.php) LAI/FPAR has been generated from NASA MODIS sensors Terra and Aqua data since 2002. We are now in dataset collection 6 (the data version to use).

    LAI is defined as the one-sided green leaf area per unit ground area in broadleaf canopies and as half the total needle surface area per unit ground area in coniferous canopies. FPAR is the fraction of photosynthetically active radiation (400-700 nm) absorbed by green vegetation. Both variables are used for calculating surface photosynthesis, evapotranspiration, and net primary production, which in turn are used to calculate terrestrial energy, carbon, water cycle processes, and biogeochemistry of vegetation. Algorithm refinements have improved quality of retrievals and consistency with field measurements over all biomes, with a focus on woody vegetation.
    
We use such data to map and understand about the dynamics of terrestrial vegetation / carbon, for example, for climate studies.

The raster data are arranged in tiles, indexed by row and column, to cover the globe:


![MODIS tiles](https://www.researchgate.net/profile/J_Townshend/publication/220473201/figure/fig5/AS:277546596880390@1443183673583/The-global-MODIS-Sinusoidal-tile-grid.png)


### Exercise

The pattern on the tile names is `hXXvYY` where `XX` is the horizontal coordinate and `YY` the vertical.


* use the map above to work out the names of the two tiles that we will need to access data over the UK
* set the variable `tiles` to contain these two names in a list

For example, for the two tiles covering Madagascar, we would set:

    tiles = ['h22v10','h22v11']


## 3.2.2  Accessing NASA MODIS URLs

Although you can access MODIS datasets through the [NASA Earthdata](https://urs.earthdata.nasa.gov/home) interface, there are many occasions that we would want to just automatically pull datasets. This is particularly true when you want a time series of data that might involve many files. For example, for analysing LAI or other variables over space/time) we will want to write code that pulls the time series of data. 

This is also something you will need to do the your assessed practical.

If the data we want to use are accessible to us as a URL, we can use the python package [`requests`](https://www.tutorialspoint.com/downloading-files-from-web-using-python).

Sometimes, we will be able to specify the parameters of the dataset we want, e.g. using [JSON](https://www.json.org). At othertimes (as in the case here) we might need to do a little work ourselves to construct the particular URL we want.

If you visit the site [https://e4ftl01.cr.usgs.gov/MOTA/MCD15A3H.006](https://e4ftl01.cr.usgs.gov/MOTA/MCD15A3H.006), you will see 'date' style links (e.g. `2018.09.30`) through to sub-directories. 

In these, e.g. [https://e4ftl01.cr.usgs.gov/MOTA/MCD15A3H.006/2018.09.30/](https://e4ftl01.cr.usgs.gov/MOTA/MCD15A3H.006/2018.09.30/) you will find URLs of a set of files. 

The files pointed to by the URLs are the MODIS MOD15 4-day composite 500 m LAI/FPAR product [MCD15A3H](https://lpdaac.usgs.gov/dataset_discovery/modis/modis_products_table/mcd15a3h_v006).

There are links to several datasets on the page, including 'quicklook files' that are jpeg format images of the datasets, e.g.:

![MCD15A3H.A2018273.h17v03](https://e4ftl01.cr.usgs.gov/MOTA/MCD15A3H.006/2018.09.30/BROWSE.MCD15A3H.A2018273.h17v03.006.2018278143630.1.jpg)

as well as `xml` files and `hdf` datasets. 



### date

The URL we have used above, [https://e4ftl01.cr.usgs.gov/MOTA/MCD15A3H.006/2018.09.30/](https://e4ftl01.cr.usgs.gov/MOTA/MCD15A3H.006/2018.09.30/) starts with a call to the server directory `MOTA`, so we can think of `https://e4ftl01.cr.usgs.gov/MOTA` as the base level URL.

The rest of the directory information `MCD15A3H.006/2018.09.30` tells us:

* the product name `MCD15A3H`
* the product version `006`
* the date of the dataset `2018.09.30`

There are several ways we could specify the date information. The most 'human readable' is probably `YYYY.MM.DD` as given here. 

## MODIS filename format

If we vist the link [https://e4ftl01.cr.usgs.gov/MOTA/MCD15A3H.006/2018.09.30/](https://e4ftl01.cr.usgs.gov/MOTA/MCD15A3H.006/2018.09.30/), we see some files that have the suffix `hdf`.

The `hdf` filenames are of the form:

    MCD15A3H.A2018273.h35v10.006.2018278143650.hdf
    
where:

* the first field (`MCD15A3H`) gives the product code
* the second (`A2018273`) gives the observation date: day of year `273`, `2018` here
* the third (`h35v10`) gives the 'MODIS tile' code for the data location
* the remaining fields specify the product version number (`006`) and a code representing the processing date.

If we want a particular dataset, we would assume then that we know the information to construct the first four fields.

We then have the task remaining of finding an address of the pattern:

    MCD15A3H.A2018273.h17v03.006.*.hdf
    
where `*` represents a wildcard (unknown element of the URL/filename).


In [26]:
from pathlib import Path
import os
import requests
import shutil
'''
Get the world borders shapefile that we will need

Python 3.5+:

'''
def get_world( borders_url = "http://thematicmapping.org/downloads",\
               file="TM_WORLD_BORDERS-0.3.zip",\
               data='data',force=False):
  '''
  get borders shapefile and download to data
  '''
  tm_borders_url = borders_url + '/' + file
  ofile = data + '/' + file
  # mkdir
  Path(data).mkdir(parents=True, exist_ok=True)

  if (not Path(ofile).exists()) or force:
    try:
      r = requests.get(tm_borders_url)
      with open(ofile, 'wb') as fp:
        fp.write (r.content)

      shutil.unpack_archive(ofile,extract_dir=data)
      return ofile
    except:
      return None
  return ofile

shp = get_world(force=True)

In [35]:
borders_url = "http://thematicmapping.org/downloads"
file="TM_WORLD_BORDERS-0.3.zip"
data='data'
force=True
tm_borders_url = borders_url + '/' + file
ofile = data + '/' + file
Path(data).mkdir(parents=True, exist_ok=True)

if (not Path(ofile).exists()) or force:
    try:
      r = requests.get(tm_borders_url)
      with open(ofile, 'wb') as fp:
        fp.write (r.content)
    except:
        pass

In [39]:
tm_borders_url = 'http://thematicmapping.org/downloads/TM_WORLD_BORDERS-0.3.zip'

r = requests.get(tm_borders_url)

In [40]:
r.content

b'<head><title>Not Acceptable!</title></head><body><h1>Not Acceptable!</h1><p>An appropriate representation of the requested resource could not be found on this server. This error was generated by Mod_Security.</p></body></html>'

## Download

Now we have some appreciation of the MODIS filename format, we can use some methods in `uclgeog_msc` to download some eaxmple datasets:

In [5]:
# by inspecting the map above
tiles = ['h17v03','h17v04']

In [16]:
from uclgeog_msc.get_modis_files import get_modis_files
from uclgeog_msc.process_timeseries \
        import create_gdal_friendly_names,find_mcdfiles,mosaic_and_clip
import matplotlib.pylab as plt
import gdal
# some libraries we need


# UK tiles
tiles = ['h17v03', 'h18v03']
# specify day of year (DOY) and year
doy,year = 1+10*8,2020

In [30]:
# download files
files = get_modis_files(doy,year,tiles,verbose=True,\
                        base_url='https://e4ftl01.cr.usgs.gov/MOTA/')
mfiles = find_mcdfiles(year,doy,tiles,'data')

fname = mosaic_and_clip(tiles,
                    doy,
                    year,
                    folder="data/",
                    layer="Lai_500m",
                    shpfile="data/TM_WORLD_BORDERS-0.3.shp",
                    country_code="LU")



print(fname)

MCD15A3H.A2020081.h17v03.006.2020086034006.hdf
data/MCD15A3H.A2020081.h17v03.006.2020086034006.hdf exists
MCD15A3H.A2020081.h18v03.006.2020086033548.hdf
data/MCD15A3H.A2020081.h18v03.006.2020086033548.hdf exists
failed to warp ['HDF4_EOS:EOS_GRID:"data/MCD15A3H.A2020081.h17v03.006.2020086034006.hdf":MOD_Grid_MCD15A3H:Lai_500m', 'HDF4_EOS:EOS_GRID:"data/MCD15A3H.A2020081.h18v03.006.2020086033548.hdf":MOD_Grid_MCD15A3H:Lai_500m'] 2020, 81, ['h17v03', 'h18v03'], data/
None
